In [35]:
import os
import numpy as np
import json
import pandas as pd
from tqdm import tqdm_notebook as tqdm
from pprint import pprint

In [36]:
path = "valori_alimenti"

In [37]:
names = np.array([
    name.split(".json")[0] for path, dirs, names in os.walk(path)
    for name in names if name.endswith(".json")
])

In [41]:
sezioni_uniche = set()

for name in tqdm(names):
    
    with open("{path}/{name}.json".format(path=path, name=names[0]), "r") as f:
        dic = json.load(f)
        
    sezioni_uniche |= set(dic.keys())
    
sezioni_uniche -= set(["name","meta-data"])
    
print(sezioni_uniche)


{'Minerali', 'Grassi polinsaturi', 'Vitamine', 'Zuccheri', 'Principali', 'Grassi saturi', 'Lipidi', 'Grassi monoinsaturi', 'Aminoacidi'}


In [43]:
chiavi_uniche = set()

for name in tqdm(names):
    
    with open("{path}/{name}.json".format(path=path, name=names[0]), "r") as f:
        dic = json.load(f)
        
    for sezione in sezioni_uniche:
        chiavi_uniche |= set(dic[sezione].keys())

chiavi_uniche |= set(["name"])
print(chiavi_uniche)


{'Zinco', 'Acido Pantotenico (Vit. B5)', 'Vitamina D (D2+D3)', 'Grassi Polinsaturi 18:2 indifferenziato', 'Serina', 'Acidi grassi, monoinsaturi', 'Istidina', 'Grassi Saturi 17:0', 'Treonina', 'Acido glutammico', 'Tiamina (Vit. B1)', 'Vitamina E, aggiunta', 'Grassi Monoinsaturi 20:1', 'Grassi Saturi 22:0', 'Tirosina', 'Rame', 'Folato alimentare', 'Fruttosio', 'Fibre', 'Sodio', 'Colina totale (Vit. J)', 'Metionina', 'Grassi Saturi 16:0', 'Folato, DFE', 'Magnesio', 'Piridossina (Vit. B6)', 'Manganese', 'Lisina', 'Triptofano', 'Colecalcifenolo (Vit. D)', 'Lattosio', 'Grassi Saturi 12:0', 'Criptoxantina, beta', 'Carotene, alfa', '22:6 n-3', 'Luteina + zeaxantina', 'Destrosio', 'Grassi Saturi 20:0', 'Cistina', 'Fenilalanina', 'Calorie', 'Grassi Polinsaturi 18:4', 'Grassi Saturi 24:0', 'Grassi Monoinsaturi 18:1 indifferenziato', 'Niacina (Vit. B3)', 'Acidi grassi, polinsaturi', 'Vitamina B-12, aggiunta', 'Grassi Monoinsaturi 14:1', 'Riboflavina (Vit. B2)', 'Fillochinone (Vit. K)', 'Valina', 

In [46]:
merged = pd.DataFrame(columns=chiavi_uniche)

for name in tqdm(names):
    df = pd.DataFrame(columns=chiavi_uniche)
    
    with open("{path}/{name}.json".format(path=path, name=name), "r") as f:
        dic = json.load(f)
        
    for sezione in sezioni_uniche:
        sub_section = dic.get(sezione,None)
        
        if sub_section:
            for sub_key,value in sub_section.items():
                unit = value["unit"]
                amount = value["amount"]
                df[sub_key + " | " + unit] = [amount]
                
    df["name"] = [dic["name"].replace("_"," ").lower().strip()]
    
    merged = merged.append(df,sort=True)
    
merged = merged.set_index("name") # set name as the index of the DataFrame

In [47]:
merged

,20:5 n-3,20:5 n-3 | g,21:5 | g,22:5 n-3,22:5 n-3 | g,22:6 n-3,22:6 n-3 | g,4:0,4:0 | g,6:0,...,Vitamina D (D2+D3),Vitamina D (D2+D3) | mcg,Vitamina D2 | mcg,Vitamina D3 | mcg,"Vitamina E, aggiunta","Vitamina E, aggiunta | mg",Zinco,Zinco | mg,Zuccheri,Zuccheri | g
name,,,,,,,,,,,,,,,,,,,,,
"riso, chicchi lunghi, parboiled, cotto",NaN,0,NaN,NaN,0,NaN,0,NaN,0,NaN,...,NaN,0,NaN,NaN,NaN,0,NaN,0.37,NaN,0.11
knorr vellutata di carciofi di gerusalemme,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
astice,NaN,0.104,NaN,NaN,0.01,NaN,0.038,NaN,0,NaN,...,NaN,0,NaN,0,NaN,0,NaN,1.3,NaN,0
spezzatino in scatola,NaN,0,0,NaN,0,NaN,0,NaN,0,NaN,...,NaN,0,NaN,NaN,NaN,0,NaN,1,NaN,1.76
"crescione, lesso",NaN,0,NaN,NaN,0,NaN,0,NaN,0,NaN,...,NaN,0,NaN,NaN,NaN,0,NaN,0.15,NaN,3.11
knorr vellutata di verdure dolci,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
nestle - motta kids - liquì,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
loacker cremkakao classic 45 grammi,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ferrero kinder colazione più - pezzo singolo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21.5


In [48]:
merged.to_csv("csv/valori_alimentari.csv")